# Word Embeddings and Sentiment

In [10]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## Tokenize the dataset

Tokenize the dataset, including padding and OOV

In [11]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

In [12]:
padded

array([[ 26,  68,   7, ...,   0,   0,   0],
       [ 20,  89,  76, ...,   0,   0,   0],
       [ 19,  13,   2, ...,   0,   0,   0],
       ...,
       [  4,  53, 809, ...,   0,   0,   0],
       [ 14,  20,  13, ...,   0,   0,   0],
       [  4,   1,   5, ...,   0,   0,   0]])

### Train a Sentiment Model with Embeddings

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 6)                 9606      
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [5]:
num_epochs = 10
history=model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 15s 154ms/step - loss: 0.6923 - accuracy: 0.5185 - val_loss: 0.7006 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6866 - accuracy: 0.5223 - val_loss: 0.7017 - val_accuracy: 0.4110
Epoch 3/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6708 - accuracy: 0.5681 - val_loss: 0.6668 - val_accuracy: 0.6917
Epoch 4/10
50/50 [==============================] - 0s 3ms/step - loss: 0.5992 - accuracy: 0.7709 - val_loss: 0.6075 - val_accuracy: 0.7018
Epoch 5/10
50/50 [==============================] - 0s 3ms/step - loss: 0.4701 - accuracy: 0.8619 - val_loss: 0.5444 - val_accuracy: 0.7419
Epoch 6/10
50/50 [==============================] - 0s 3ms/step - loss: 0.3520 - accuracy: 0.9065 - val_loss: 0.4912 - val_accuracy: 0.7619
Epoch 7/10
50/50 [==============================] - 0s 3ms/step - loss: 0.2664 - accuracy: 0.9341 - val_loss: 0.4769 - val_accuracy: 0.7594
Epoch 8/10
50/50 

## Get files for visualizing the network

Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox.

In [6]:
# First get the weights of the embedding layer

e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


## Predicting Sentiment in New Reviews

In [9]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)               

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')


['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']
1/1 [==============================] - 0s 16ms/step
I love this phone
[0.98903626]


I hate spaghetti
[0.08008483]


Everything was cold
[0.3097954]


Everything was hot exactly as I wanted
[0.5898766]


Everything was green
[0.50313115]


the host seated us immediately
[0.7616487]


they gave us free chocolate cake
[0.92002845]


not sure about the wilted flowers on the table
[0.04934696]


only works when I stand on tippy toes
[0.96331614]


does not work when I stand on my head
[0.00796461]


